# Next Word Prediction using RNN's

In [6]:
import tensorflow as tf
print(tf.__version__)

2.8.0


Importing Libraries and Data

In [7]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
import pickle
import numpy as np
import os

In [8]:
file = open("C:\\Users\\Parth Salke\\Downloads\\Datasets\\metamorphosis-clean.txt", "r", encoding = "utf8")
lines = []

In [9]:
for i in file:
    lines.append(i)
    
print("The First Line: ", lines[0])
print("The Last Line: ", lines[-1])

The First Line:  ﻿One morning, when Gregor Samsa woke from troubled dreams, he found

The Last Line:  first to get up and stretch out her young body.


Cleaning the Data

In [10]:
data = ""

for i in lines:
    data = ' '. join(lines)
    
data = data.replace('\n', '').replace('\r', '').replace('\ufeff', '')
data[:360]

'One morning, when Gregor Samsa woke from troubled dreams, he found himself transformed in his bed into a horrible vermin.  He lay on his armour-like back, and if he lifted his head a little he could see his brown belly, slightly domed and divided by arches into stiff sections.  The bedding was hardly able to cover it and seemed ready to slide off any moment.'

In [11]:
import string

translator = str.maketrans(string.punctuation, ' '*len(string.punctuation)) #map punctuation to space
new_data = data.translate(translator)

new_data[:500]

'One morning  when Gregor Samsa woke from troubled dreams  he found himself transformed in his bed into a horrible vermin   He lay on his armour like back  and if he lifted his head a little he could see his brown belly  slightly domed and divided by arches into stiff sections   The bedding was hardly able to cover it and seemed ready to slide off any moment   His many legs  pitifully thin compared with the size of the rest of him  waved about helplessly as he looked    What s happened to me   he'

In [12]:
z = []

for i in data.split():
    if i not in z:
        z.append(i)
        
data = ' '.join(z)
data[:500]

'One morning, when Gregor Samsa woke from troubled dreams, he found himself transformed in his bed into a horrible vermin. He lay on armour-like back, and if lifted head little could see brown belly, slightly domed divided by arches stiff sections. The bedding was hardly able to cover it seemed ready slide off any moment. His many legs, pitifully thin compared with the size of rest him, waved about helplessly as looked. "What\'s happened me?" thought. It wasn\'t dream. room, proper human room altho'

Labeling

In [13]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])

# saving the tokenizer for predict function.
pickle.dump(tokenizer, open('tokenizer1.pkl', 'wb'))

sequence_data = tokenizer.texts_to_sequences([data])[0]
sequence_data[:10]

[17, 53, 293, 2, 18, 729, 135, 730, 294, 8]

In [14]:
vocab_size = len(tokenizer.word_index) + 1
print(vocab_size)

2617


In [15]:
sequences = []

for i in range(1, len(sequence_data)):
    words = sequence_data[i-1:i+1]
    sequences.append(words)
    
print("The Length of sequences are: ", len(sequences))
sequences = np.array(sequences)
sequences[:10]

The Length of sequences are:  3889


array([[ 17,  53],
       [ 53, 293],
       [293,   2],
       [  2,  18],
       [ 18, 729],
       [729, 135],
       [135, 730],
       [730, 294],
       [294,   8],
       [  8, 731]])

In [16]:
X = []
y = []

for i in sequences:
    X.append(i[0])
    y.append(i[1])
    
X = np.array(X)
y = np.array(y)

In [17]:
print("The Data is: ", X[:5])
print("The responses are: ", y[:5])

The Data is:  [ 17  53 293   2  18]
The responses are:  [ 53 293   2  18 729]


In [18]:
y = to_categorical(y, num_classes=vocab_size)
y[:5]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

Creating RNN/LSTM Model

In [19]:
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=1))
model.add(LSTM(1000, return_sequences=True))
model.add(LSTM(1000))
model.add(Dense(1000, activation="relu"))
model.add(Dense(vocab_size, activation="softmax"))

In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 1, 10)             26170     
                                                                 
 lstm (LSTM)                 (None, 1, 1000)           4044000   
                                                                 
 lstm_1 (LSTM)               (None, 1000)              8004000   
                                                                 
 dense (Dense)               (None, 1000)              1001000   
                                                                 
 dense_1 (Dense)             (None, 2617)              2619617   
                                                                 
Total params: 15,694,787
Trainable params: 15,694,787
Non-trainable params: 0
_________________________________________________________________


In [23]:
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import TensorBoard

checkpoint = ModelCheckpoint("nextword1.h5", monitor='loss', verbose=1,
    save_best_only=True, mode='auto')

reduce = ReduceLROnPlateau(monitor='loss', factor=0.2, patience=3, min_lr=0.0001, verbose = 1)

logdir='logsnextword1'
tensorboard_Visualization = TensorBoard(log_dir=logdir)

In [25]:
model.compile(loss="categorical_crossentropy", optimizer=Adam(learning_rate=0.001))

In [26]:
model.fit(X, y, epochs=150, batch_size=64, callbacks=[checkpoint, reduce, tensorboard_Visualization])

Epoch 1/150
61/61 [==============================] - ETA: 0s - loss: 7.8753
Epoch 1: loss improved from inf to 7.87530, saving model to nextword1.h5
61/61 [==============================] - 11s 141ms/step - loss: 7.8753 - lr: 0.0010
Epoch 2/150
61/61 [==============================] - ETA: 0s - loss: 7.8629
Epoch 2: loss improved from 7.87530 to 7.86287, saving model to nextword1.h5
61/61 [==============================] - 9s 142ms/step - loss: 7.8629 - lr: 0.0010
Epoch 3/150
61/61 [==============================] - ETA: 0s - loss: 7.8268
Epoch 3: loss improved from 7.86287 to 7.82678, saving model to nextword1.h5
61/61 [==============================] - 9s 146ms/step - loss: 7.8268 - lr: 0.0010
Epoch 4/150
61/61 [==============================] - ETA: 0s - loss: 7.6802
Epoch 4: loss improved from 7.82678 to 7.68016, saving model to nextword1.h5
61/61 [==============================] - 11s 178ms/step - loss: 7.6802 - lr: 0.0010
Epoch 5/150
61/61 [==============================] - ETA: 

61/61 [==============================] - 9s 144ms/step - loss: 3.3316 - lr: 0.0010
Epoch 36/150
61/61 [==============================] - ETA: 0s - loss: 3.2204
Epoch 36: loss improved from 3.33161 to 3.22039, saving model to nextword1.h5
61/61 [==============================] - 9s 141ms/step - loss: 3.2204 - lr: 0.0010
Epoch 37/150
61/61 [==============================] - ETA: 0s - loss: 3.1109
Epoch 37: loss improved from 3.22039 to 3.11089, saving model to nextword1.h5
61/61 [==============================] - 9s 143ms/step - loss: 3.1109 - lr: 0.0010
Epoch 38/150
61/61 [==============================] - ETA: 0s - loss: 3.0691
Epoch 38: loss improved from 3.11089 to 3.06914, saving model to nextword1.h5
61/61 [==============================] - 9s 155ms/step - loss: 3.0691 - lr: 0.0010
Epoch 39/150
61/61 [==============================] - ETA: 0s - loss: 2.9922
Epoch 39: loss improved from 3.06914 to 2.99223, saving model to nextword1.h5
61/61 [==============================] - 11s 178

61/61 [==============================] - ETA: 0s - loss: 1.6951
Epoch 70: loss improved from 1.73643 to 1.69514, saving model to nextword1.h5
61/61 [==============================] - 8s 138ms/step - loss: 1.6951 - lr: 0.0010
Epoch 71/150
61/61 [==============================] - ETA: 0s - loss: 1.6773
Epoch 71: loss improved from 1.69514 to 1.67729, saving model to nextword1.h5
61/61 [==============================] - 8s 137ms/step - loss: 1.6773 - lr: 0.0010
Epoch 72/150
61/61 [==============================] - ETA: 0s - loss: 1.7006
Epoch 72: loss did not improve from 1.67729
61/61 [==============================] - 8s 138ms/step - loss: 1.7006 - lr: 0.0010
Epoch 73/150
61/61 [==============================] - ETA: 0s - loss: 1.6729
Epoch 73: loss improved from 1.67729 to 1.67285, saving model to nextword1.h5
61/61 [==============================] - 8s 139ms/step - loss: 1.6729 - lr: 0.0010
Epoch 74/150
61/61 [==============================] - ETA: 0s - loss: 1.6347
Epoch 74: loss imp

61/61 [==============================] - 9s 142ms/step - loss: 1.1614 - lr: 0.0010
Epoch 106/150
61/61 [==============================] - ETA: 0s - loss: 1.1444
Epoch 106: loss improved from 1.16136 to 1.14440, saving model to nextword1.h5
61/61 [==============================] - 9s 145ms/step - loss: 1.1444 - lr: 0.0010
Epoch 107/150
61/61 [==============================] - ETA: 0s - loss: 1.1266
Epoch 107: loss improved from 1.14440 to 1.12664, saving model to nextword1.h5
61/61 [==============================] - 9s 144ms/step - loss: 1.1266 - lr: 0.0010
Epoch 108/150
61/61 [==============================] - ETA: 0s - loss: 1.1242
Epoch 108: loss improved from 1.12664 to 1.12423, saving model to nextword1.h5
61/61 [==============================] - 9s 144ms/step - loss: 1.1242 - lr: 0.0010
Epoch 109/150
61/61 [==============================] - ETA: 0s - loss: 1.1154
Epoch 109: loss improved from 1.12423 to 1.11535, saving model to nextword1.h5
61/61 [==============================] -

61/61 [==============================] - 9s 153ms/step - loss: 0.6386 - lr: 1.0000e-04
Epoch 141/150
61/61 [==============================] - ETA: 0s - loss: 0.6384
Epoch 141: loss improved from 0.63860 to 0.63835, saving model to nextword1.h5
61/61 [==============================] - 9s 153ms/step - loss: 0.6384 - lr: 1.0000e-04
Epoch 142/150
61/61 [==============================] - ETA: 0s - loss: 0.6383
Epoch 142: loss improved from 0.63835 to 0.63832, saving model to nextword1.h5
61/61 [==============================] - 9s 153ms/step - loss: 0.6383 - lr: 1.0000e-04
Epoch 143/150
61/61 [==============================] - ETA: 0s - loss: 0.6385
Epoch 143: loss did not improve from 0.63832
61/61 [==============================] - 9s 153ms/step - loss: 0.6385 - lr: 1.0000e-04
Epoch 144/150
61/61 [==============================] - ETA: 0s - loss: 0.6379
Epoch 144: loss improved from 0.63832 to 0.63788, saving model to nextword1.h5
61/61 [==============================] - 11s 174ms/step - 

# Predictions code:

In [33]:
# Importing the Libraries

from tensorflow.keras.models import load_model
import numpy as np
import pickle

# Load the model and tokenizer

model = load_model('nextword1.h5')
tokenizer = pickle.load(open('tokenizer1.pkl', 'rb'))

def Predict_Next_Words(model, tokenizer, text):
    """
        In this function we are using the tokenizer and models trained
        and we are creating the sequence of the text entered and then
        using our model to predict and return the the predicted word.
    
    """
    for i in range(3):
        sequence = tokenizer.texts_to_sequences([text])[0]
        sequence = np.array(sequence)
        
        preds = model.predict_classes(sequence)
        #print(preds)
        predicted_word = ""
        
        for key, value in tokenizer.word_index.items():
            if value == preds:
                predicted_word = key
                break
        
        print(predicted_word)
        return predicted_word

In [34]:
"""
    We are testing our model and we will run the model
    until the user decides to stop the script.
    While the script is running we try and check if 
    the prediction can be made on the text. If no
    prediction can be made we just continue.

"""

# text1 = "at the dull"
# text2 = "collection of textile"
# text3 = "what a strenuous"
# text4 = "stop the script"

while(True):

    text = input("Enter your line: ")
    
    if text == "stop the script":
        print("Ending The Program.....")
        break
    
    else:
        try:
            text = text.split(" ")
            text = text[-1]

            text = ''.join(text)
            Predict_Next_Words(model, tokenizer, text)
            
        except:
            continue

Enter your line: at your service
Enter your line: at the dull
Enter your line: of textile
Enter your line: stop the script
Ending The Program.....
